# Imports

In [11]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium import webdriver
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities

In [ ]:
# url = 'https://www.dnb.com/business-directory/company-information.health_care_and_social_assistance.ca.html'
# # firefox agent
# agent = {"User-Agent":'Mozilla/5.0 (X11; Linux x86_64; rv:97.0) Gecko/20100101 Firefox/97.0'}
# chromium = {
#     "User-Agent":'Mozilla/5.0 (X11; Linux x86_64; rv:97.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36',


# Trials

In [52]:
cap = DesiredCapabilities().FIREFOX
cap["marionette"] = False
driver = webdriver.Firefox(capabilities=cap, executable_path="/home/ada/Downloads/geckodriver")

driver.get('https://www.dnb.com/business-directory/company-information.health_care_and_social_assistance.ca.html')

# links = driver.find_elements_by_class_name("col-md-12 data")
elements = driver.find_element(By.TAG_NAME, 'div')
vals = elements.find_elements(By.CLASS_NAME, value="col-md-6")
print('length of links are: ', len(links))
for i in range(len(vals)):
    print("element in list ",vals[i].text)


driver.close()

length of links are:  0
element in list  
element in list  
element in list  
element in list  
element in list  
element in list  
element in list  
element in list  
element in list  
element in list  
element in list  
element in list  
element in list  
element in list  
element in list  
element in list  
element in list  
element in list  
element in list  
element in list  
element in list   
element in list  Alberta Health Services
element in list  We Care Health Services Gp Inc.
element in list  OMERS Private Equity Inc
element in list  The Winnipeg Regional Health Authority
element in list  Centre Intégré Universitaire Santé et Services Sociaux du Centre-Sud-de-l'Ile-de-Montréal
element in list  Provincial Health Services Authority
element in list  Young Men's Christian Association of Edmonton
element in list  Centre de Santé et de Services Sociaux de Laval
element in list  Vancouver Island Health Authority
element in list  University Health Network
element in list  Interior 